We are calculating variances and covariances as:

$$
var(\hat{y}^i_{h,t}) = \big[\sigma^2_{\alpha} + 2 \sigma_{\alpha \beta} h + \sigma^2_{\beta}h^2 \big] + var(z_{h,t}^i) +  \phi_t^2 \sigma^2_{\epsilon}
$$


$$
covar(\hat{y}^i_{h,t}, \hat{y}^i_{h+n,t+n}) = \big[\sigma^2_{\alpha} + 2 \sigma_{\alpha \beta} (h+n) + \sigma^2_{\beta}h(h+n) \big] + \rho^2 var(z_{h,t}^i) 
$$

Where $n = 1,...,min(H-h, T-t)$ is the minimum of the distance of an individual to the maximum experience and the maximum time period - e.g. an individual who is age 20 in 1995 could potentially have 45 years of experience, but as the sample is truncated in 1996, $n$ will be 1 ($T-t$), while an individual who is 60 years old in 1970 could stay in the sample for 26 more years, but as age is capped at 64, $n$ will be 4 ($H-h$).

The variance of $z_{h,t}^i$ will be obtained recursively by:

\begin{align}
var(z_{1,t}^i) &= \pi^2_t \sigma^2_{\eta}, \ h=1, t>1 \\
var(z_{h,1}^i) &= \pi^2_1 \sigma^2_{\eta} \sum_{j=0}^{h-1}\rho^{2j}, \ t=1, h>1 \\
var(z_{h,t}^i) &= \rho^2 var(z_{h-1,t-1}) + \pi_t^2 \sigma^2_{\eta}, \ t,h>1
\end{align}

Where the first line implicitly assumes that the initial value of the persistent shock is zero for all individuals (i.e. at $h=1$, the first period of working life, there is no $\rho z_0$ entering the individuals income process).

What the above equations tell us is that we will have two slightly different ways of calculating an individual's variance for any $t,h>1$, depending on whether $h \leq t$ or $h > t$. Intuitively, this is due to the fact that when $h > t$, the individual has already accumulated a history of shocks at $t=1$, the first period of observation. In this case, the recursion for $var(z_{h,t}^i)$ will end with the $var(z_{h,1}^i)$ term, so individuals will differ based on how many working years they accrued before the start of the sample (these are individuals who started work prior to 1968). If $h \leq t$, the recursion ends in $var(z_{1,t}^i)$, so individuals will only differ in the first shock they experienced upon entering the sample at the start of their working life (these are individuals who start work after 1968).

To illustrate, let's compare $var(z_{4,5})$ and $var(z_{5,4})$. In the first case, $h<t$, so we end up at the $var(z_{1,t}^i)$ term. Writing out the recursion gives us $\rho^6 \pi^2_2 \sigma^2 + \rho^4 \pi^2_3 \sigma^2 + \rho^2 \pi^2_4 \sigma^2 + \pi^2_5 \sigma^2$, or, in matrix notation:

$$
\sigma^2 \left( \begin{bmatrix} \rho^6 & \rho^4 & \rho^2 & 1 \end{bmatrix} 
\begin{bmatrix} \pi^2_2 \\ \pi^2_3 \\ \pi^2_4 \\ \pi^2_5 \end{bmatrix} \right)
$$

while in the second case, we end up at the $var(z_{h,1}^i)$ term, which will add information on the shocks accumulated during the working life prior to entering the sample. Writing out the recursion gives $\rho^6 \pi^2_1 \sigma^2 \sum_{j=0}^{h-1} \rho^{2j} + \rho^4 \pi^2_2 \sigma^2 + \rho^2 \pi^2_3 \sigma^2 + \pi^2_4 \sigma^2$, or, in matrix notation:

$$
\sigma^2 \left( \begin{bmatrix} \rho^6 & \rho^4 & \rho^2 & 1 \end{bmatrix} 
\begin{bmatrix} \pi^2_1 \sum_{j=0}^{h-1} \rho^{2j}  \\ \pi^2_2 \\ \pi^2_3 \\ \pi^2_4 \end{bmatrix} \right)
$$




In the [construction of the empirical covariance matrix](https://github.com/nilshg/psidJulia/blob/master/prepCovMat.ipynb) we are allowing for a maximum lang length of 29. The maximum age that a 4-year-cohort could reach (without considering the length of the sample) is 41 - upon entering the age-midpoint is 22 (ages 20-24), while the cohort exits the sample with an age midpoint of 62 (ages 60-64). For this reason, every cohort will have a $(41x41)$ matrix holding all possible variances and covariances for each year of life, although only a maximum of 29 values will be found in each column (the maximum lag length). 

For the cohort with an age-midpoint of 22 in 1968, this matrix would look as follows:

\begin{pmatrix}
v^1_{68} & 0           & \cdots &        & & 0 \\
c^2_{68,69} & v^2_{69} & 0      & \cdots & & \vdots \\
\vdots      &   \ddots    & \ddots &        & &  \\
            &             &        &        & &  \\
NA          & c^{29}_{69,96}& \cdots &     & v^{29}_{96} & & \\
0            &    \cdots  &        &        & &  0 \\
\vdots      &             &        &        & &  \vdots \\
0           &   \cdots    &       &         & \cdots & 0
 \end{pmatrix}

where in a slight abuse of notation we use superscripts to indicate experience ($h$) and subscripts to indicate years ($t$). As we see, the bottom and right hand parts of the matrix are left empty, as the cohort cannot reach the maximum age of 41 due to sample limitations. 

Cohorts entering the sample (i.e. having an age-midpoint of 22) after 1968 will have a similarly structured covariance matrix, with observations below the diagonal from $(1,1)$ to $(T-t, T-t)$, where $T$ is the last year of observations (1996) and $t$ the year that the cohohrt is entering. In the extreme case, a cohort entering in 1996 would just have one entry in its variance-covariance matrix, but since we are restricting the sample to cohorts with at least 20 years of observations, practically the last cohort to enter does so in 1977 (leaving 20 years to 1996). Their covariance matrix will then b:

\begin{pmatrix}
v^1_{77} & 0           & \cdots &        & & 0 \\
c^2_{77,78} & v^2_{78} & 0      & \cdots & & \vdots \\
\vdots      &   \ddots    & \ddots &        & &  \\
            &             &        &        & &  \\
c^{20}_{77,96}   & c^{20}_{78,96}& \cdots &     & v^{20}_{96} & & \\
0            &    \cdots  &        &        & &  0 \\
\vdots      &             &        &        & &  \vdots \\
0           &   \cdots    &       &         & \cdots & 0
 \end{pmatrix}
 
 so that there are only 20 non-zero off-diagonal elements.
 
 On the other hand, we will also have cohorts that were older than 22 in 1968, i.e. that entered the labor market prior to the start of our sample period. These cohorts will then have missing observations in the first rows and columns of the matrix, e.g. the cohort that was 22 in 1967:
 
\begin{pmatrix}
0        & 0       & \cdots &        & & 0 \\
0        & v^2_{68}& 0      & \cdots & & \vdots \\
\vdots   &     & \ddots &        & &  \\
         &             &        &        & &  \\
         & c^{29}_{69,96}& \cdots &     & v^{29}_{96} & & \\
0        &    \cdots  &        &        & &  0 \\
\vdots   &             &        &        & &  \vdots \\
0        &   \cdots    &       &         & \cdots & 0
 \end{pmatrix} 

Again, in the extreme we could have a cohort that was 62 in 1968, and would hence only have one observed variance (the $(41,41)$ element of the covariance matrix. And again, as we are restricting the sample to cohorts with at least 20 years of valid observations, the oldest possible cohort to remain in the sample is the one which is 43 in 1968. The covariance matrix will then be:

\begin{pmatrix}
0        & 0   &          & \cdots      &            &        &             & 0      \\
0        & 0   &          &             &            & \cdots &             & \vdots \\
\vdots   &     & \ddots   &             &            &        &             &        \\
         &     &          & v^{22}_{68} &            &        &             &        \\
         &     &          &             & v^{23}_{69}&        &             &        \\
         &     &          & \vdots      &            & \ddots &             &  \vdots \\
\vdots   &     &          &             &            &        & v^{40}_{86} &  0     \\
0        &     &          & c^{22}_{68,87} &            &        & \cdots      & v^{41}_{87}
 \end{pmatrix} 


This implies that we only have some cohorts for which we can actually observe all covariances up to the 29 lags we're allowing in the estimation; these are the cohorts entering from 1956 to 1968, while those entering after 1968 will have fewer observations as the sample ends before their retirement, while the cohorts entering before 1956 will retire before the sample ends. 

To hammer the point home, let's try a markdown table listing cohorts by entry year:

| CovN | Birth Year | Entry Year | Age 1968 | Age 1996 | Years in sample | Obs in Var-Cov Matrix |
|:----:|:----------:|:----------:|:--------:|:--------:|:---------------:|:---------------------:|
|  1   |   1955     |  1977      |  13      |   41     |     20          |        210            |
|  2   |   1954     |  1976      |  14      |   42     |     21          |        231            |
|  3   |   1953     |  1975      |  15      |   43     |     22          |        253            |
|  4   |   1952     |  1974      |  16      |   44     |     23          |        276            |
|  5   |   1951     |  1973      |  17      |   45     |     24          |        300            |
|  6   |   1950     |  1972      |  18      |   46     |     25          |        325            |
|  7   |   1949     |  1971      |  19      |   47     |     26          |        351            |
|  8   |   1948     |  1970      |  20      |   48     |     27          |        378            |
|  9   |   1947     |  1969      |  21      |   49     |     28          |        406            |
|  10  |   1946     |  1968      |  22      |   50     |     29          |        435            |
|  11  |   1945     |  1968      |  23      |   51     |     29          |        435            |
|  12  |   1944     |  1968      |  24      |   52     |     29          |        435            |
|  13  |   1943     |  1968      |  25      |   53     |     29          |        435            |
|  14  |   1942     |  1968      |  26      |   54     |     29          |        435            |
|  15  |   1941     |  1968      |  27      |   55     |     29          |        435            |
|  16  |   1940     |  1968      |  28      |   56     |     29          |        435            |
|  17  |   1939     |  1968      |  29      |   57     |     29          |        435            |
|  18  |   1938     |  1968      |  30      |   58     |     29          |        435            |
|  19  |   1937     |  1968      |  31      |   59     |     29          |        435            |
|  20  |   1936     |  1968      |  32      |   60     |     29          |        435            |
|  21  |   1935     |  1968      |  33      |   61     |     29          |        435            |
|  22  |   1934     |  1968      |  34      |   62     |     29          |        435            |
|  23  |   1933     |  1968      |  35      |   63     |     28          |        406            |
|  24  |   1932     |  1968      |  36      |   64     |     27          |        378            |
|  25  |   1931     |  1968      |  37      |   65     |     26          |        351            |
|  26  |   1930     |  1968      |  38      |   66     |     25          |        325            |
|  27  |   1929     |  1968      |  39      |   67     |     24          |        300            |
|  28  |   1928     |  1968      |  40      |   68     |     23          |        276            |
|  29  |   1927     |  1968      |  41      |   69     |     22          |        253            |
|  30  |   1926     |  1968      |  42      |   70     |     21          |        231            |
|  31  |   1925     |  1968      |  43      |   71     |     20          |        210            |
| (32) |  (1924)    | (1968)     | (44)     |  (72)    |    (19)         |       (190)           |
| (33) |  (1923)    | (1968)     | (45)     |  (73)    |    (18)         |       (171)           |

Where the last two rows technically shouldn't be in the table, as the respective cohorts spend less than 20 years in the sample - they're just listed here for completeness as Guvenen's code sets `maxcoh` at 33 and thus creates 33 variance-covariance matrices.


The calculations outlined above yield a $41x41$ matrix for each of our 31 cohorts, with every cohort matrix containing at most 29 years of valid covariances. This however is not the final object used in the minimization procedure - for this, we construct a $TxT$ matrix which holds the weighted sums of covariances for each sample year, where the weights are given by the number of observations across all cohorts present in a given year. 

Consider element $(1,1)$ of our covariance matrix, corresponding to the variance of income residuals in 1968. To calculate the element, we would have to sum the variances for all cohorts present in the sample in 1968 - that is, cohorts 10 to 31 - and weight these observations by the number of observations for each cohort. That is, the element would be calculated as

$$
c_1 = \frac{1}{\sum_{i=1}^{22} obs^i_{68}} \big[ obs^1_{68} var^1_{68} + obs^2_{68} var^2_{68} + ... + obs^{22}_{68} var^{22}_{68} \big]
$$

The problem with the above is that we have data for years 1968 to 1997, and then 1999 to 2013 in two-yearly releases. That means in total there are 38 observations rather than the 29 above, but we have to be a bit careful as individuals in the sample age two years for each observation towards the end (that is someone who e.g. has been in the sample for 19 years in 1997, but is 63 year olds at that time, will not be able to record a 20th observation, as he is 65 during the next survey in 1999).

| CovN | Birth Year | Entry Year | Age 1968 | Age 2013 | Years in sample | Observations in Var-Cov Matrix         |
|:----:|:----------:|:----------:|:--------:|:--------:|:---------------:|:---------------------:|
|  1   |   1964     |  1986      |   4      |   49     |     28          |        20             |
|  2   |   1963     |  1985      |   5      |   50     |     29          |        21             |
|  3   |   1962     |  1984      |   6      |   51     |     30          |        22             |
|  4   |   1961     |  1983      |   7      |   52     |     31          |        23             |
|  5   |   1960     |  1982      |   8      |   53     |     32          |        24             |
|  6   |   1959     |  1981      |   9      |   54     |     33          |        25             |
|  7   |   1958     |  1980      |  10      |   55     |     34          |        26             |
|  8   |   1957     |  1979      |  11      |   56     |     35          |        27             |
|  9   |   1956     |  1978      |  12      |   57     |     36          |        28             |
|  10  |   1955     |  1977      |  13      |   58     |     37          |        29             |
|  11  |   1954     |  1976      |  14      |   59     |     38          |        30             |
|  12  |   1953     |  1975      |  15      |   60     |     39          |        31             |
|  13  |   1952     |  1974      |  16      |   61     |     40          |        32             |
|  14  |   1951     |  1973      |  17      |   62     |     41          |        33             |
|  15  |   1950     |  1972      |  18      |   63     |     41          |        33             |
|  16  |   1949     |  1971      |  19      |   64     |     41          |        34             |
|  17  |   1948     |  1970      |  20      |   65     |     41          |        34             |
|  18  |   1947     |  1969      |  21      |   66     |     41          |        35             |
|  19  |   1946     |  1968      |  22      |   67     |     41          |        35             |
|  20  |   1945     |  1968      |  23      |   68     |     40          |        35             |
|  21  |   1944     |  1968      |  24      |   69     |     39          |        34             |
|  22  |   1943     |  1968      |  25      |   70     |     38          |        34             |
|  23  |   1942     |  1968      |  26      |   71     |     37          |        33             |
|  24  |   1941     |  1968      |  27      |   72     |     36          |        33             |
|  25  |   1940     |  1968      |  28      |   73     |     35          |        32             |
|  26  |   1939     |  1968      |  29      |   74     |     34          |        32             |
|  27  |   1938     |  1968      |  30      |   75     |     33          |        31             |
|  28  |   1937     |  1968      |  31      |   76     |     32          |        31             |
|  29  |   1936     |  1968      |  32      |   77     |     31          |        30             |
|  30  |   1935     |  1968      |  33      |   78     |     30          |        30             |
|  31  |   1934     |  1968      |  34      |   79     |     29          |        29             |
|  32  |   1933     |  1968      |  35      |   80     |     28          |        28             |
|  33  |   1932     |  1968      |  36      |   81     |     27          |        27             |
|  34  |   1931     |  1968      |  37      |   82     |     26          |        26             |
|  35  |   1930     |  1968      |  38      |   83     |     25          |        25             |
|  36  |   1929     |  1968      |  39      |   84     |     24          |        24             |
|  37  |   1928     |  1968      |  40      |   85     |     23          |        23             |
|  38  |   1927     |  1968      |  41      |   86     |     22          |        22             |
|  39  |   1926     |  1968      |  42      |   87     |     21          |        21             |
|  40  |   1925     |  1968      |  43      |   88     |     20          |        20             |

Let's think about the subperiod 1968-1979. When using short subperiods, we will only require 8 years of valid observations, which of course means there are more (older) cohort present in the original sample. We have:

| CovN | Birth Year | Entry Year | Age 1968 | Age 1979 | Years in sample | Observations          |
|:----:|:----------:|:----------:|:--------:|:--------:|:---------------:|:---------------------:|
|  1   |   1950     |  1972      |  18      |   29     |     8           |        33             |
|  2   |   1949     |  1971      |  19      |   30     |     9           |        34             |
|  3   |   1948     |  1970      |  20      |   31     |     10          |        34             |
|  4   |   1947     |  1969      |  21      |   32     |     11          |        35             |
|  5   |   1946     |  1968      |  22      |   33     |     12          |        78             |
|  6   |   1945     |  1968      |  23      |   34     |     12          |        35             |
|  7   |   1944     |  1968      |  24      |   35     |     12          |        34             |
|  8   |   1943     |  1968      |  25      |   36     |     12          |        34             |
|  9   |   1942     |  1968      |  26      |   37     |     12          |        33             |
|  10  |   1941     |  1968      |  27      |   38     |     12          |        33             |
|  11  |   1940     |  1968      |  28      |   39     |     12          |        32             |
|  12  |   1939     |  1968      |  29      |   40     |     12          |        32             |
|  13  |   1938     |  1968      |  30      |   41     |     12          |        31             |
|  14  |   1937     |  1968      |  31      |   42     |     12          |        31             |
|  15  |   1936     |  1968      |  32      |   43     |     12          |        30             |
|  16  |   1935     |  1968      |  33      |   44     |     12          |        30             |
|  17  |   1934     |  1968      |  34      |   45     |     12          |        29             |
|  18  |   1933     |  1968      |  35      |   46     |     12          |        28             |
|  19  |   1932     |  1968      |  36      |   47     |     12          |        27             |
|  20  |   1931     |  1968      |  37      |   48     |     12          |        26             |
|  21  |   1930     |  1968      |  38      |   49     |     12          |        25             |
|  22  |   1929     |  1968      |  39      |   50     |     12          |        24             |
|  23  |   1928     |  1968      |  40      |   51     |     12          |        23             |
|  24  |   1927     |  1968      |  41      |   52     |     12          |        22             |
|  25  |   1926     |  1968      |  42      |   53     |     12          |        21             |
|  26  |   1925     |  1968      |  43      |   54     |     12          |        20             |
|  27  |   1924     |  1968      |  44      |   55     |     12          |        20             |
|  28  |   1923     |  1968      |  45      |   56     |     12          |        20             |
|  29  |   1922     |  1968      |  46      |   57     |     12          |        20             |
|  30  |   1921     |  1968      |  47      |   58     |     12          |        20             |
|  31  |   1920     |  1968      |  48      |   59     |     12          |        20             |
|  32  |   1919     |  1968      |  49      |   60     |     12          |        20             |
|  33  |   1918     |  1968      |  50      |   61     |     12          |        20             |
|  34  |   1917     |  1968      |  51      |   62     |     12          |        20             |
|  35  |   1916     |  1968      |  52      |   63     |     12          |        20             |
|  36  |   1915     |  1968      |  53      |   64     |     12          |        20             |
|  37  |   1914     |  1968      |  54      |   65     |     11          |        20             |
|  38  |   1913     |  1968      |  55      |   66     |     10          |        20             |
|  39  |   1912     |  1968      |  56      |   67     |     9           |        20             |
|  40  |   1911     |  1968      |  57      |   68     |     8           |        20             |

What about 1980 to 1991? 

| CovN | Birth Year | Entry Year | Age 1980 | Age 1991 | Years in sample | Observations          |
|:----:|:----------:|:----------:|:--------:|:--------:|:---------------:|:---------------------:|
|  1   |   1962     |  1984      |  18      |   29     |     8           |        33             |
|  2   |   1961     |  1983      |  19      |   30     |     9           |        34             |
|  3   |   1960     |  1982      |  20      |   31     |     10          |        34             |
|  4   |   1959     |  1981      |  21      |   32     |     11          |        35             |
|  5   |   1958     |  1980      |  22      |   33     |     12          |        78             |
|  6   |   1957     |  1980      |  23      |   34     |     12          |        35             |
|  7   |   1956     |  1980      |  24      |   35     |     12          |        34             |
|  8   |   1955     |  1980      |  25      |   36     |     12          |        34             |
|  9   |   1954     |  1980      |  26      |   37     |     12          |        33             |
|  10  |   1953     |  1980      |  27      |   38     |     12          |        33             |
|  11  |   1952     |  1980      |  28      |   39     |     12          |        32             |
|  12  |   1951     |  1980      |  29      |   40     |     12          |        32             |
|  13  |   1950     |  1980      |  30      |   41     |     12          |        31             |
|  14  |   1949     |  1980      |  31      |   42     |     12          |        31             |
|  15  |   1948     |  1980      |  32      |   43     |     12          |        30             |
|  16  |   1947     |  1980      |  33      |   44     |     12          |        30             |
|  17  |   1946     |  1980      |  34      |   45     |     12          |        29             |
|  18  |   1945     |  1980      |  35      |   46     |     12          |        28             |
|  19  |   1944     |  1980      |  36      |   47     |     12          |        27             |
|  20  |   1943     |  1980      |  37      |   48     |     12          |        26             |
|  21  |   1942     |  1980      |  38      |   49     |     12          |        25             |
|  22  |   1941     |  1980      |  39      |   50     |     12          |        24             |
|  23  |   1940     |  1980      |  40      |   51     |     12          |        23             |
|  24  |   1939     |  1980      |  41      |   52     |     12          |        22             |
|  25  |   1938     |  1980      |  42      |   53     |     12          |        21             |
|  26  |   1937     |  1980      |  43      |   54     |     12          |        20             |
|  27  |   1936     |  1980      |  44      |   55     |     12          |        20             |
|  28  |   1935     |  1980      |  45      |   56     |     12          |        20             |
|  29  |   1934     |  1980      |  46      |   57     |     12          |        20             |
|  30  |   1933     |  1980      |  47      |   58     |     12          |        20             |
|  31  |   1932     |  1980      |  48      |   59     |     12          |        20             |
|  32  |   1931     |  1980      |  49      |   60     |     12          |        20             |
|  33  |   1930     |  1980      |  50      |   61     |     12          |        20             |
|  34  |   1929     |  1980      |  51      |   62     |     12          |        20             |
|  35  |   1928     |  1980      |  52      |   63     |     11          |        20             |
|  36  |   1927     |  1980      |  53      |   64     |     10          |        20             |
|  37  |   1926     |  1980      |  54      |   65     |     9           |        20             |
|  38  |   1925     |  1980      |  55      |   66     |     8           |        20             |

And finally, 1992 to 2013:

| CovN | Birth Year | Entry Year | Age 1992 | Age 2013 | Years in sample | Observations          |
|:----:|:----------:|:----------:|:--------:|:--------:|:---------------:|:---------------------:|
|  1   |   1977     |  1999      |  15      |   36     |     15 (8)      |        120 (36)       |
|  2   |   1976     |  1998      |  16      |   37     |     16 (8)      |        136 (36)       |
|  3   |   1975     |  1997      |  17      |   38     |     17 (9)      |        153 (55)       |
|  4   |   1974     |  1996      |  18      |   39     |     18 (10)     |        253 (105)      |
|  5   |   1973     |  1995      |  19      |   40     |     19 (11)     |        253 (105)      |
|  6   |   1972     |  1994      |  20      |   41     |     20 (12)     |        253 (105)      |
|  7   |   1971     |  1993      |  21      |   42     |     21 (13)     |        253 (105)      |
|  8   |   1970     |  1992      |  22      |   43     |     22 (14)     |        253 (105)      |
|  9   |   1969     |  1992      |  23      |   44     |     22 (14)     |        253 (105)      |
|  10  |   1968     |  1992      |  24      |   45     |     22 (14)     |        253 (105)      |
|  11  |   1967     |  1992      |  25      |   46     |     22 (14)     |        253 (105)      |
|  12  |   1966     |  1992      |  26      |   47     |     22 (14)     |        253 (105)      |
|  13  |   1965     |  1992      |  27      |   48     |     22 (14)     |        253 (105)      |
|  14  |   1964     |  1992      |  28      |   49     |     22 (14)     |        253 (105)      |
|  15  |   1963     |  1992      |  29      |   50     |     22 (14)     |        253 (105)      |
|  16  |   1962     |  1992      |  30      |   51     |     22 (14)     |        253 (105)      |
|  17  |   1961     |  1992      |  31      |   52     |     22 (14)     |        253 (105)      |
|  18  |   1960     |  1992      |  32      |   53     |     22 (14)     |        253 (105)      |
|  19  |   1959     |  1992      |  33      |   54     |     22 (14)     |        253 (105)      |
|  20  |   1958     |  1992      |  34      |   55     |     22 (14)     |        253 (105)      |
|  21  |   1957     |  1992      |  35      |   56     |     22 (14)     |        253 (105)      |
|  22  |   1956     |  1978      |  36      |   57     |     22 (14)     |        253 (105)      |
|  23  |   1955     |  1977      |  37      |   58     |     22 (14)     |        253 (105)      |
|  24  |   1954     |  1976      |  38      |   59     |     22 (14)     |        253 (105)      |
|  25  |   1953     |  1975      |  39      |   60     |     22 (14)     |        253 (105)      |
|  26  |   1952     |  1968      |  40      |   61     |     22 (14)     |        253 (105)      |
|  27  |   1951     |  1968      |  41      |   62     |     22 (14)     |        253 (105)      |
|  28  |   1950     |  1968      |  42      |   63     |     21 (13)     |        231 (91)       |
|  29  |   1949     |  1968      |  43      |   64     |     20 (13)     |        210 (91)       |
|  30  |   1948     |  1968      |  44      |   65     |     19 (12)     |        190 (78)       |
|  31  |   1947     |  1968      |  45      |   66     |     18 (12)     |        171 (78)       |
|  32  |   1946     |  1968      |  46      |   67     |     17 (11)     |        153  (66)      |
|  33  |   1945     |  1968      |  47      |   68     |     16 (11)     |        136  (66)      |
|  34  |   1944     |  1968      |  48      |   69     |     15 (10)     |        120  (55)      |
|  35  |   1943     |  1968      |  49      |   70     |     14 (10)     |        105  (55)      |
|  36  |   1942     |  1968      |  50      |   71     |     13 (9)      |        91  (45)       |
|  37  |   1941     |  1968      |  51      |   72     |     12 (9)      |        78 (45)        |
|  38  |   1940     |  1968      |  52      |   73     |     11 (8)      |        66 (36)        |
|  39  |   1939     |  1992      |  53      |   75     |     10 (8)      |        55 (36)        |